# First Download Data using api:
https://www.kaggle.com/discussions/general/74235

In [ ]:
 !pip install -q kaggle

In [ ]:
 from google.colab import files
 # upload json file:
 files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jackparkinson","key":"fab3b257b9f0ef42e6a66b63e329f43e"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d gpiosenka/100-bird-species

Dataset URL: https://www.kaggle.com/datasets/gpiosenka/100-bird-species
License(s): CC0-1.0
100-bird-species.zip: Skipping, found more recently modified local copy (use --force to force download)


## Set up paths

In [ ]:
import zipfile
import os

In [ ]:
zip_path = '/content/100-bird-species.zip'

# directory to unzip (can change if you want)
extract_to = '/content/birds_dataset'

# removes any current directory with name
if os.path.exists(extract_to):
    !rm -rf {extract_to}

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Unzipping completed!")

Unzipping completed!


In [ ]:
train_dir = '/content/birds_dataset/train'
test_dir = '/content/birds_dataset/test'
valid_dir = '/content/birds_dataset/valid'

# counts files
def count_images(directory):
    count = 0
    for _, _, files in os.walk(directory):
        count += len(files)
    return count

train_count = count_images(train_dir)
valid_count = count_images(valid_dir)
test_count = count_images(test_dir)
train_split = train_count / (train_count+test_count+valid_count)
valid_split = valid_count / (train_count+test_count+valid_count)
test_split = test_count / (train_count+test_count+valid_count)


print(f"Number of images in Train: {train_count} with split: {train_split}")
print(f"Number of images in Valid: {valid_count} with split: {valid_split}")
print(f"Number of images in Test: {test_count} with split: {test_split}")

Number of images in Train: 84635 with split: 0.9415920342660066
Number of images in Valid: 2625 with split: 0.029203982866996717
Number of images in Test: 2625 with split: 0.029203982866996717


## Move around images so its roughly 80/10/10 split:
Can manually specify a different split by choosing num_test_files and num_valid_files variables

In [ ]:
import os
import shutil

def adjust_dataset_split(train_dir, test_dir, valid_dir, num_test_files, num_valid_files):
    # iterates through each species directory in the train directory
    for species in os.listdir(train_dir):
        species_train_dir = os.path.join(train_dir, species)
        species_test_dir = os.path.join(test_dir, species)
        species_valid_dir = os.path.join(valid_dir, species)

        # train directory files for each species
        files = os.listdir(species_train_dir)

        # ensures test and valid directories exist
        os.makedirs(species_test_dir, exist_ok=True)
        os.makedirs(species_valid_dir, exist_ok=True)

        # move files to test
        for file in files[:num_test_files]:
            shutil.move(os.path.join(species_train_dir, file), os.path.join(species_test_dir, file))

        # update list of files remaining in train
        remaining_files = os.listdir(species_train_dir)

        # move files to valid
        for file in remaining_files[:num_valid_files]:
            shutil.move(os.path.join(species_train_dir, file), os.path.join(species_valid_dir, file))


# specify number of files to move to respective dirs
num_test_files = 10
num_valid_files = 10


adjust_dataset_split(train_dir, test_dir, valid_dir, num_test_files, num_valid_files)
print("Files have been moved to achieve the specified dataset split.")


Files have been moved to achieve the specified dataset split.


In [ ]:
train_count = count_images(train_dir)
valid_count = count_images(valid_dir)
test_count = count_images(test_dir)
train_split = train_count / (train_count+test_count+valid_count)
valid_split = valid_count / (train_count+test_count+valid_count)
test_split = test_count / (train_count+test_count+valid_count)


print(f"Number of images in Train: {train_count} with proportion: {train_split}")
print(f"Number of images in Valid: {valid_count} with proportion: {valid_split}")
print(f"Number of images in Test: {test_count} with proportion: {test_split}")

Number of images in Train: 74135 with proportion: 0.8247761027980197
Number of images in Valid: 7875 with proportion: 0.08761194860099016
Number of images in Test: 7875 with proportion: 0.08761194860099016
